# Disclaimer
En este script se realizará la fase de modelado de la problemática, teniendo en cuenta tanto diferentes posibles modelos, como técnicas (ensamble, redes neuronales, etc.)

In [1]:
pip install jinja2 markupsafe

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Imports generales
import pandas as pd
import io
import numpy as np
import statistics
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import cluster
import sklearn.metrics
from sklearn.cluster import DBSCAN
from sklearn.neighbors import kneighbors_graph
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score

from sklearn.metrics import davies_bouldin_score

In [3]:
path = 'C:/Users/Usuario/Documents/TFM/'


In [4]:
base = pd.read_excel(path + 'Base_modelado_nueva.xlsx')
mpios = pd.read_excel( path + 'DNP mpios2025 base mpios.xlsx', sheet_name = 'Municipios')

famd = pd.read_csv( path + 'FAMD_municipios.csv', sep = ';')
embed = pd.read_excel( path + 'Embeding_MPIO_ANIO_ajustados_otros.xlsx')


base

,Unnamed: 0,ENTIDAD,ESTADO,CLASIFICACION,SEXO,EDAD_GRUPO,EDAD_QUINQ,CICLO_VIT,EST_CIVIL,ESCOLARIDAD,...,FECHA_DES,ANIO_DES,MES_DES,DIA_DES,PAIS_DES,DEPTO,DEPTO_DES,MPIO,MPIO_DES,ZONA_DES
0,0,Instituto Nacional de Medicina Legal y Ciencia...,Desaparecido,SIN_INFO,Hombre,Mayores_edad,(20 a 24),Juventud,Soltero,basica_secundaria,...,2006-12-29,2006,diciembre,viernes,Colombia,11,BogotÃ¡ D.C.,11001,BogotÃ¡ D.C.,cabec_mpal
1,1,Instituto Nacional de Medicina Legal y Ciencia...,Desaparecido,SIN_INFO,Hombre,Mayores_edad,(20 a 24),Juventud,Soltero,basica_secundaria,...,2006-12-26,2006,diciembre,martes,Colombia,11,BogotÃ¡ D.C.,11001,BogotÃ¡ D.C.,cabec_mpal
2,2,Instituto Nacional de Medicina Legal y Ciencia...,Desaparecido,SIN_INFO,Mujer,a) Menores de Edad (<18 aÃ±os),(10 a 14),Adolescencia,Soltero,basica_secundaria,...,2006-12-29,2006,diciembre,viernes,Colombia,11,BogotÃ¡ D.C.,11001,BogotÃ¡ D.C.,cabec_mpal
3,3,Instituto Nacional de Medicina Legal y Ciencia...,Aparecio vivo,SIN_INFO,Mujer,a) Menores de Edad (<18 aÃ±os),(15 a 17),Adolescencia,Soltero,Educacion_superior,...,2006-12-29,2006,diciembre,viernes,Colombia,11,BogotÃ¡ D.C.,11001,BogotÃ¡ D.C.,cabec_mpal
4,4,Instituto Nacional de Medicina Legal y Ciencia...,Aparecio vivo,SIN_INFO,Hombre,Mayores_edad,(70 a 74),Adulto_mayor,Union_libre,primaria,...,2006-12-31,2006,diciembre,domingo,Colombia,11,BogotÃ¡ D.C.,11001,BogotÃ¡ D.C.,cabec_mpal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188975,190901,Unidad de BÃºsqueda de Personas Desaparecidas,Desaparecido,SIN_INFO,Hombre,Mayores_edad,(25 a 29),Juventud,Soltero,primaria,...,2007-01-01,2007,enero,lunes,Colombia,95,Guaviare,95001,San JosÃ© Del Guaviare,Rural
188976,190902,Unidad de BÃºsqueda de Personas Desaparecidas,Desaparecido,SIN_INFO,Hombre,Mayores_edad,(18 a 19),Juventud,Soltero,primaria,...,2011-11-09,2011,noviembre,miÃ©rcoles,Colombia,5,Antioquia,5250,El Bagre,cabec_mpal
188977,190903,Unidad de BÃºsqueda de Personas Desaparecidas,Desaparecido,SIN_INFO,Hombre,Mayores_edad,(20 a 24),Juventud,Casado,primaria,...,2001-01-01,2001,enero,lunes,Colombia,68,Santander,68235,El Carmen De Chucuri,SIN_INFO
188978,190904,Unidad de BÃºsqueda de Personas Desaparecidas,Desaparecido,DES_FORZ,Mujer,Mayores_edad,(25 a 29),Juventud,Soltero,inicial y preescolar,...,1984-01-01,1984,enero,domingo,Colombia,68,Santander,68235,El Carmen De Chucuri,SIN_INFO


In [5]:
mpios['Tipología_2025'] = mpios['Tipología_2025'].astype(str)

mpios = mpios[['MPIO', 'CAT_ruralidad 2023']]

In [6]:
base['ANIO_MPIO'] = base['MPIO'].astype(str) + '_' + base['ANIO_DES'].astype(str)

In [7]:
base['ANIO_DES'] = base['ANIO_DES'].astype(str)

In [8]:
base = base.merge(mpios, on = 'MPIO', how = 'left')

In [9]:
base['CAT_ruralidad 2023'] = base['CAT_ruralidad 2023'].fillna('SIN_INFO')


In [10]:
reemplazos = { 
              'Aparecio vivo': 1,
              'Aparecio muerto': 1,
              'Desaparecido': 0}

base['ESTADO'] = base['ESTADO'].replace(reemplazos)

reemplazos = { 
              'basica_secundaria': 'Secundaria',
              'secundaria': 'Secundaria',
              'primaria': 'Primaria',
              'inicial y preescolar': 'Primaria',
              'profesional': 'Avanzada',
              'Universitario': 'Avanzada',
              'master o especializacion': 'Avanzada',
              'doctorado': 'Avanzada',
              'Sin escolaridad': 'Sin escolaridad',
              'SIN_INFO': 'SIN_INFO'}

base['ESCOLARIDAD'] = base['ESCOLARIDAD'].replace(reemplazos)


reemplazos = { 
              'Aparecio vivo': 'Aparecio',
              'Aparecio muerto': 'Aparecio',
              'Desaparecido': 'Desaparecido'}

base['ESTADO'] = base['ESTADO'].replace(reemplazos)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_34448\1551899384.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  base['ESTADO'] = base['ESTADO'].replace(reemplazos)


In [11]:
#base = base[base['ANIO_DES']>= "2019"]

In [12]:
base.columns

Index(['Unnamed: 0', 'ENTIDAD', 'ESTADO', 'CLASIFICACION', 'SEXO',
       'EDAD_GRUPO', 'EDAD_QUINQ', 'CICLO_VIT', 'EST_CIVIL', 'ESCOLARIDAD',
       'PRT_GRUP', 'FECHA_DES', 'ANIO_DES', 'MES_DES', 'DIA_DES', 'PAIS_DES',
       'DEPTO', 'DEPTO_DES', 'MPIO', 'MPIO_DES', 'ZONA_DES', 'ANIO_MPIO',
       'CAT_ruralidad 2023'],
      dtype='object')

In [13]:
famd

,MUNICIPIO_ANIO,0,1,2,3,4
0,11001_1948,8.332948,2.274413,4.164528,13.417492,6.112574
1,11001_1949,8.332948,2.274413,4.164528,13.417492,6.112574
2,11001_1951,8.332948,2.274413,4.164528,13.417492,6.112574
3,11001_1954,8.332948,2.274413,4.164528,13.417492,6.112574
4,11001_1955,8.352830,2.290133,4.173245,13.427974,6.096916
...,...,...,...,...,...,...
19770,99773_2016,-0.870946,3.214372,-2.963620,0.676791,-0.349189
19771,99773_2018,-0.908679,3.183656,-2.979574,0.660528,-0.315992
19772,99773_2021,-0.892859,3.197927,-2.973816,0.661608,-0.335413
19773,99773_2023,-0.892859,3.197927,-2.973816,0.661608,-0.335413


In [14]:
import pandas as pd
import numpy as np

# Supongamos que tienes
# Base: tu DataFrame original con columnas 'MPIO' y 'ESTADO'
# a: Serie con el conteo de desapariciones por municipio
a = base.groupby('ANIO_MPIO')['ESTADO'].count()

# Opcional: revisa cuántos hay por municipio
counts = a

# Umbral de desapariciones
threshold = 20

# Identifica aquellos municipios con conteo menor que el umbral
mpios_menores = counts[counts < threshold].index

# Ahora creas una nueva columna, por ej. 'MPIO_reclasificado'
base['MPIO_ANIO_reclasificado'] = np.where(
    base['ANIO_MPIO'].isin(mpios_menores),
    'Otros',
    base['ANIO_MPIO']
)


# nombres = ['MUNICIPIO_ANIO', 'FAMD_0', 'FAMD_1', 'FAMD_2', 'FAMD_3', 'FAMD_4']
# famd.columns =  nombres

nombres = ['MUNICIPIO_ANIO', 'MPIO_ANIO_reclasificado', 'MPIO_ANIO_ID',
       'MPIO_ANIO_emb_0', 'MPIO_ANIO_emb_1', 'MPIO_ANIO_emb_2',
       'MPIO_ANIO_emb_3', 'MPIO_ANIO_emb_4', 'MPIO_ANIO_emb_5',
       'MPIO_ANIO_emb_6', 'MPIO_ANIO_emb_7', 'MPIO_ANIO_emb_8',
       'MPIO_ANIO_emb_9', 'MPIO_ANIO_emb_10', 'MPIO_ANIO_emb_11',
       'MPIO_ANIO_emb_12', 'MPIO_ANIO_emb_13', 'MPIO_ANIO_emb_14']
embed.columns =  nombres

### Cruce con FAMD

In [15]:
# base = base.merge(famd, right_on='MUNICIPIO_ANIO', left_on= 'ANIO_MPIO', how = 'left')

## Cruce con Embedding

In [16]:
famd

,MUNICIPIO_ANIO,0,1,2,3,4
0,11001_1948,8.332948,2.274413,4.164528,13.417492,6.112574
1,11001_1949,8.332948,2.274413,4.164528,13.417492,6.112574
2,11001_1951,8.332948,2.274413,4.164528,13.417492,6.112574
3,11001_1954,8.332948,2.274413,4.164528,13.417492,6.112574
4,11001_1955,8.352830,2.290133,4.173245,13.427974,6.096916
...,...,...,...,...,...,...
19770,99773_2016,-0.870946,3.214372,-2.963620,0.676791,-0.349189
19771,99773_2018,-0.908679,3.183656,-2.979574,0.660528,-0.315992
19772,99773_2021,-0.892859,3.197927,-2.973816,0.661608,-0.335413
19773,99773_2023,-0.892859,3.197927,-2.973816,0.661608,-0.335413


Cruzamos con el FAMD

In [17]:
nombres = ['MUNICIPIO_ANIO', 'FAMD_0', 'FAMD_1', 'FAMD_2', 'FAMD_3', 'FAMD_4']
famd.columns =  nombres

In [18]:
base = base.merge(famd, right_on='MUNICIPIO_ANIO', left_on= 'ANIO_MPIO', how = 'left')

base = base.fillna(0)

Cruzamos con el embedding

In [19]:
embed = embed.rename(columns={'MUNICIPIO_ANIO': 'MPIO_ANIO_EMBED'})

In [20]:
embed.drop_duplicates(subset = 'MPIO_ANIO_EMBED',inplace = True)
base = base.merge(embed, right_on='MPIO_ANIO_EMBED', left_on= 'ANIO_MPIO', how = 'left')

# 🧾 Evaluación de columnas para el modelo de predicción (`ESTADO`)

| Columna         | Acción         |
|-----------------|----------------|
| `Unnamed: 0`    | ❌ **Quitar**   |
| `ID`            | ✅ **Pasa**     |
| `ENTIDAD`       | ❌ **Quitar**   |
| `ESTADO`        | 🎯 **Predecir** |
| `CLASIFICACION` | ✅ **Pasa**     |
| `SEXO`          | ✅ **Pasa**     |
| `EDAD_GRUPO`    | ❌ **Quitar**   |
| `EDAD_QUINQ`    | ❌ **Quitar**   |
| `CICLO_VIT`     | ✅ **Pasa**     |
| `EST_CIVIL`     | ✅ **Pasa**     |
| `ESCOLARIDAD`   | ✅ **Pasa**     |
| `PRT_GRUP`      | ❌ **Quitar**   |
| `FECHA_DES`     | ❌ **Quitar**   |
| `ANIO_DES`      | ✅ **Pasa**     |
| `MES_DES`       | ✅ **Pasa**     |
| `DIA_DES`       | ❌ **Quitar**   |
| `PAIS_DES`      | ❌ **Quitar**   |
| `DEPTO`         | ❌ **Quitar**   |
| `DEPTO_DES`     | ❌ **Quitar**   |
| `MPIO`          | ✅ **Pasa**     |
| `MPIO_DES`      | ❌ **Quitar**   |
| `ZONA_DES`      | ❌ **Quitar**   |
| `ANIO_MPIO`     | ❓ **Revisar**  |

---

### 🔍 Notas:

- 🎯 `ESTADO` es la **variable objetivo** (target).
- ✅ Columnas **seleccionadas** para el modelo.
- ❌ Columnas descartadas por ser irrelevantes, administrativas o redundantes.
- ❓ `ANIO_MPIO`: si es una llave compuesta (`municipio_año`), Depende del modelo, pasa o no



In [21]:
base.columns

Index(['Unnamed: 0', 'ENTIDAD', 'ESTADO', 'CLASIFICACION', 'SEXO',
       'EDAD_GRUPO', 'EDAD_QUINQ', 'CICLO_VIT', 'EST_CIVIL', 'ESCOLARIDAD',
       'PRT_GRUP', 'FECHA_DES', 'ANIO_DES', 'MES_DES', 'DIA_DES', 'PAIS_DES',
       'DEPTO', 'DEPTO_DES', 'MPIO', 'MPIO_DES', 'ZONA_DES', 'ANIO_MPIO',
       'CAT_ruralidad 2023', 'MPIO_ANIO_reclasificado_x', 'MUNICIPIO_ANIO',
       'FAMD_0', 'FAMD_1', 'FAMD_2', 'FAMD_3', 'FAMD_4', 'MPIO_ANIO_EMBED',
       'MPIO_ANIO_reclasificado_y', 'MPIO_ANIO_ID', 'MPIO_ANIO_emb_0',
       'MPIO_ANIO_emb_1', 'MPIO_ANIO_emb_2', 'MPIO_ANIO_emb_3',
       'MPIO_ANIO_emb_4', 'MPIO_ANIO_emb_5', 'MPIO_ANIO_emb_6',
       'MPIO_ANIO_emb_7', 'MPIO_ANIO_emb_8', 'MPIO_ANIO_emb_9',
       'MPIO_ANIO_emb_10', 'MPIO_ANIO_emb_11', 'MPIO_ANIO_emb_12',
       'MPIO_ANIO_emb_13', 'MPIO_ANIO_emb_14'],
      dtype='object')

In [23]:
columnas_a_dejar = [
'ESTADO', 'CLASIFICACION', 'SEXO', 'CICLO_VIT', 'EST_CIVIL',
       'ESCOLARIDAD', 'CAT_ruralidad 2023', 'MPIO_ANIO_emb_0',
       'MPIO_ANIO_emb_1', 'MPIO_ANIO_emb_2', 'MPIO_ANIO_emb_3',
       'MPIO_ANIO_emb_4', 'MPIO_ANIO_emb_5', 'MPIO_ANIO_emb_6',
       'MPIO_ANIO_emb_7', 'MPIO_ANIO_emb_8', 'MPIO_ANIO_emb_9',
       'MPIO_ANIO_emb_10', 'MPIO_ANIO_emb_11', 'MPIO_ANIO_emb_12',
       'MPIO_ANIO_emb_13', 'MPIO_ANIO_emb_14'
]

Base_em = base[columnas_a_dejar]

columnas_a_dejar = [
'ESTADO', 'CLASIFICACION', 'SEXO', 'CICLO_VIT', 'EST_CIVIL',
       'ESCOLARIDAD', 'CAT_ruralidad 2023','FAMD_0', 
       'FAMD_1', 'FAMD_2', 'FAMD_3', 'FAMD_4'
]

Base_famd = base[columnas_a_dejar]

## Fase de modelado

## Empezamos la fase de modelado 

Ya con la base lista, empezaré a modelar la problemática con diferentes técnicas
Inicialmente:

- K vecinos más cercanos
- Random Forest
- XGBoost
- Logístico

In [24]:
from sklearn.model_selection import train_test_split
# EMBED
# # X: todas las columnas excepto 'ESTADO'
X = Base_em.drop(columns='ESTADO')

# y: variable objetivo
y = Base_em['ESTADO']

# Dividir en entrenamiento y prueba
X_train_em, X_test_em, y_train_em, y_test_em = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

#FAMD

# # X: todas las columnas excepto 'ESTADO'
X = Base_famd.drop(columns='ESTADO').fillna(0)

# y: variable objetivo
y = Base_famd['ESTADO']

# Dividir en entrenamiento y prueba
X_train_famd, X_test_famd, y_train_famd, y_test_famd = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)



In [27]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    classification_report, confusion_matrix
)
from xgboost import XGBClassifier

# --- Helper OHE compatible con distintas versiones de sklearn ---
def _ohe_compatible():
    try:
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore", sparse=False)

# === 1) Selección de columnas para el preprocesado (usamos TODO lo que esté en X_train_em) ===
cat_cols = X_train_em.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = [c for c in X_train_em.columns if c not in cat_cols]

pre = ColumnTransformer(
    transformers=[
        ("cat", _ohe_compatible(), cat_cols) if len(cat_cols) else ("cat", "drop", []),
        ("num", "passthrough", num_cols)    if len(num_cols) else ("num", "drop", [])
    ],
    remainder="drop"
)

# === 2) XGBoost con EXACTAMENTE tus hiperparámetros ===
xgb = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    tree_method="hist",   # usa 'gpu_hist' si tienes GPU
    random_state=42,
    n_jobs=-1,
    n_estimators=393,
    max_depth=6,
    learning_rate=0.0019845516353496095,
    subsample=0.8816822115019555,
    colsample_bytree=0.7830186052470381,
    min_child_weight=1,
    gamma=1.8585385818109046e-07,
    reg_alpha=0.5002018067635705,
    reg_lambda=0.021523888540223697,
    scale_pos_weight=1.82578080395847
)

pipe = Pipeline(steps=[("pre", pre), ("classifier", xgb)])

# === 3) Entrenar ===
pipe.fit(X_train_em, y_train_em)

# === 4) Predecir (clase y probas) ===
y_pred  = pipe.predict(X_test_em)
y_proba = pipe.predict_proba(X_test_em)[:, 1]

# (Opcional) Métricas rápidas estándar
print("Accuracy :", accuracy_score(y_test_em, y_pred))
print("Recall(y=1):", recall_score(y_test_em, y_pred))
print("Precision(y=1):", precision_score(y_test_em, y_pred))
print("F1(y=1)  :", f1_score(y_test_em, y_pred))
print("AUC      :", roc_auc_score(y_test_em, y_proba))
print("\nMatriz de confusión:\n", confusion_matrix(y_test_em, y_pred))
print("\nClassification report:\n", classification_report(y_test_em, y_pred, digits=4))



Accuracy : 0.6995272868381134
Recall(y=1): 0.7840523267425604
Precision(y=1): 0.5526695526695526
F1(y=1)  : 0.648335088045251
AUC      : 0.7977919918278159

Matriz de confusión:
 [[23956 12710]
 [ 4325 15703]]

Classification report:
               precision    recall  f1-score   support

           0     0.8471    0.6534    0.7377     36666
           1     0.5527    0.7841    0.6483     20028

    accuracy                         0.6995     56694
   macro avg     0.6999    0.7187    0.6930     56694
weighted avg     0.7431    0.6995    0.7061     56694



In [25]:
# Revisar valores faltantes en Base_em
print("=== NAs en Base_em ===")
print(Base_em.isna().sum())

# Revisar valores faltantes en Base_famd
print("\n=== NAs en Base_famd ===")
print(Base_famd.isna().sum())

# Opcional: resumen total de NAs por dataset
print("\nResumen:")
print(f"Total NAs en Base_em   : {Base_em.isna().sum().sum()}")
print(f"Total NAs en Base_famd : {Base_famd.isna().sum().sum()}")

=== NAs en Base_em ===
ESTADO                0
CLASIFICACION         0
SEXO                  0
CICLO_VIT             0
EST_CIVIL             0
ESCOLARIDAD           0
CAT_ruralidad 2023    0
MPIO_ANIO_emb_0       0
MPIO_ANIO_emb_1       0
MPIO_ANIO_emb_2       0
MPIO_ANIO_emb_3       0
MPIO_ANIO_emb_4       0
MPIO_ANIO_emb_5       0
MPIO_ANIO_emb_6       0
MPIO_ANIO_emb_7       0
MPIO_ANIO_emb_8       0
MPIO_ANIO_emb_9       0
MPIO_ANIO_emb_10      0
MPIO_ANIO_emb_11      0
MPIO_ANIO_emb_12      0
MPIO_ANIO_emb_13      0
MPIO_ANIO_emb_14      0
dtype: int64

=== NAs en Base_famd ===
ESTADO                0
CLASIFICACION         0
SEXO                  0
CICLO_VIT             0
EST_CIVIL             0
ESCOLARIDAD           0
CAT_ruralidad 2023    0
FAMD_0                0
FAMD_1                0
FAMD_2                0
FAMD_3                0
FAMD_4                0
dtype: int64

Resumen:
Total NAs en Base_em   : 0
Total NAs en Base_famd : 0


## Logistica

In [148]:
# ============================================================
# RandomizedSearchCV - Regresión Logística con múltiples sets
# ============================================================
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    accuracy_score, recall_score, precision_score, f1_score
)
from scipy.stats import loguniform

RANDOM_STATE = 42
CV_SPLITS = 5
N_ITER = 40  # puedes subir si tienes más tiempo/recursos
SCORING = {
    "recall": "recall_macro",
    "f1": "f1_macro",
    "accuracy": "accuracy",
    "roc_auc": "roc_auc_ovr"
}
REFIT = "recall"  # prioriza recall_macro

def build_var_sets(X):
    """
    Define combinaciones de variables según lo que exista en X.
    - DEMOS: columnas categóricas demográficas si existen
    - EMB_*: columnas que empiezan por 'MPIO_ANIO_emb_'
    - FAMD_*: columnas que empiezan por 'FAMD_'
    - combinaciones mixtas
    """
    cols = X.columns
    demos_candidates = [
        'CLASIFICACION','SEXO','CICLO_VIT','EST_CIVIL','ESCOLARIDAD','CAT_ruralidad 2023'
    ]
    DEMOS = [c for c in demos_candidates if c in cols]
    EMB = [c for c in cols if c.startswith('MPIO_ANIO_emb_')]
    FAMD = [c for c in cols if c.startswith('FAMD_')]

    var_sets = {}
    if DEMOS:
        var_sets["DEMOS"] = DEMOS
    if EMB:
        var_sets["EMB_full"] = EMB
        for k in [5, 8, 12, 15]:
            if len(EMB) >= k:
                var_sets[f"EMB_top{k}"] = EMB[:k]
    if FAMD:
        var_sets["FAMD_full"] = FAMD
        for k in [2, 3, 4, 5]:
            if len(FAMD) >= k:
                var_sets[f"FAMD_top{k}"] = FAMD[:k]

    if DEMOS and EMB:
        var_sets["DEMOS+EMB_full"] = DEMOS + EMB
        if len(EMB) >= 8:
            var_sets["DEMOS+EMB_top8"] = DEMOS + EMB[:8]
    if DEMOS and FAMD:
        var_sets["DEMOS+FAMD_full"] = DEMOS + FAMD
        if len(FAMD) >= 3:
            var_sets["DEMOS+FAMD_top3"] = DEMOS + FAMD[:3]

    if not var_sets:
        var_sets["ALL"] = list(cols)

    return var_sets

def make_search():
    """Crea el objeto RandomizedSearchCV con pipeline y espacio de hiperparámetros."""
    # Placeholders: dtypes reales se fijan al ajustarse el ColumnTransformer dinámicamente
    pre = ColumnTransformer(transformers=[], remainder="drop")
    logreg = LogisticRegression(max_iter=5000, random_state=RANDOM_STATE)

    pipe = Pipeline(steps=[
        ("pre", pre),
        ("clf", logreg)
    ])

    param_distributions = {
        "clf__C": loguniform(1e-3, 1e2),
        "clf__penalty": ["l1", "l2"],
        "clf__solver": ["liblinear", "saga"],  # ambos soportan l1/l2 (liblinear: OvR)
        "clf__class_weight": [None, "balanced"]
    }
    cv = StratifiedKFold(n_splits=CV_SPLITS, shuffle=True, random_state=RANDOM_STATE)

    search = RandomizedSearchCV(
        estimator=pipe,
        param_distributions=param_distributions,
        n_iter=N_ITER,
        scoring=SCORING,
        refit=REFIT,
        cv=cv,
        n_jobs=-1,
        random_state=RANDOM_STATE,
        verbose=0,
        return_train_score=False
    )
    return search

def fit_with_features(search, X_train, y_train, feat_list):
    """Ajusta el pipeline con ColumnTransformer construido a partir de feat_list."""
    Xsub = X_train[feat_list].copy()

    cat_cols = Xsub.select_dtypes(include=['object','category']).columns.tolist()
    num_cols = [c for c in Xsub.columns if c not in cat_cols]

    # --- OneHotEncoder compatible con sklearn 1.0–1.5+ ---
    try:
        # sklearn >= 1.2
        ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        # sklearn < 1.2
        ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

    transformers = []
    if cat_cols:
        transformers.append(("cat", ohe, cat_cols))
    if num_cols:
        transformers.append(("num", StandardScaler(), num_cols))

    # Si por alguna razón no hubiera ni cat ni num (no debería pasar), hacemos passthrough
    if not transformers:
        transformers.append(("passthrough_all", "passthrough", Xsub.columns.tolist()))

    pre = ColumnTransformer(transformers=transformers, remainder="drop")

    # Insertamos el preprocess en el pipeline del search
    search.estimator.set_params(pre=pre)

    # Ajuste
    search.fit(Xsub, y_train)
    return search

def evaluate_on_test(best_estimator, X_test, y_test, feat_list):
    Xsub_test = X_test[feat_list].copy()
    y_pred = best_estimator.predict(Xsub_test)

    # AUC
    auc = None
    try:
        if hasattr(best_estimator, "predict_proba"):
            proba = best_estimator.predict_proba(Xsub_test)
            auc = roc_auc_score(y_test, proba[:,1]) if proba.shape[1] == 2 \
                  else roc_auc_score(y_test, proba, multi_class="ovr")
    except Exception:
        pass

    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred, digits=4)

    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "recall_macro": recall_score(y_test, y_pred, average="macro"),
        "precision_macro": precision_score(y_test, y_pred, average="macro"),
        "f1_macro": f1_score(y_test, y_pred, average="macro"),
        "auc": auc,
        "confusion_matrix": cm,
        "classification_report": report
    }
    return metrics

def run_experiments(dataset_name, X_train, y_train, X_test, y_test):
    results_rows = []
    var_sets = build_var_sets(X_train)

    for combo_name, feat_list in var_sets.items():
        search = make_search()
        search = fit_with_features(search, X_train, y_train, feat_list)
        best_est = search.best_estimator_

        test_metrics = evaluate_on_test(best_est, X_test, y_test, feat_list)

        # --- Print detallado por combinación ---
        print("="*110)
        print(f"[{dataset_name}] COMBINACIÓN: {combo_name}")
        print(f"Features ({len(feat_list)}): {feat_list}")
        print("\n>> Hiperparámetros óptimos (CV, refit=recall_macro):")
        print(search.best_params_)
        print(f"Mejor score CV (recall_macro): {search.best_score_:.4f}\n")

        print(">> MÉTRICAS EN TEST:")
        print(f"Accuracy        : {test_metrics['accuracy']:.4f}")
        print(f"Recall (macro)  : {test_metrics['recall_macro']:.4f}")
        print(f"Precision (macro): {test_metrics['precision_macro']:.4f}")
        print(f"F1 (macro)      : {test_metrics['f1_macro']:.4f}")
        print(f"AUC (OvR/binario): {None if test_metrics['auc'] is None else round(test_metrics['auc'], 4)}\n")

        print("Matriz de confusión (rows=verdad, cols=pred):")
        print(test_metrics["confusion_matrix"], "\n")

        print("Classification report (TEST):")
        print(test_metrics["classification_report"])
        print()

        # --- Guardar fila resumen ---
        results_rows.append({
            "dataset": dataset_name,
            "combo": combo_name,
            "n_features": len(feat_list),
            "features": feat_list,
            "best_params": search.best_params_,
            "cv_best_recall_macro": search.best_score_,
            "test_accuracy": test_metrics["accuracy"],
            "test_recall_macro": test_metrics["recall_macro"],
            "test_precision_macro": test_metrics["precision_macro"],
            "test_f1_macro": test_metrics["f1_macro"],
            "test_auc": test_metrics["auc"]
        })

    results_df = pd.DataFrame(results_rows).sort_values(
        by=["dataset", "test_recall_macro", "test_f1_macro"], ascending=[True, False, False]
    ).reset_index(drop=True)

    print("\n" + "#"*110)
    print(f"TOP combinaciones por dataset {dataset_name} (ordenado por recall_macro/f1_macro en TEST):")
    display_cols = ["dataset","combo","n_features","cv_best_recall_macro",
                    "test_accuracy","test_recall_macro","test_precision_macro","test_f1_macro","test_auc"]
    print(results_df[display_cols].head(10).to_string(index=False))
    print("#"*110 + "\n")

    return results_df

# =========================
# Ejecutar para ambos sets
# =========================
all_results = []

# 1) EMBEDDINGS
res_em = run_experiments(
    dataset_name="Base_em",
    X_train=X_train_em, y_train=y_train_em,
    X_test=X_test_em,   y_test=y_test_em
)
all_results.append(res_em)

# 2) FAMD
res_famd = run_experiments(
    dataset_name="Base_famd",
    X_train=X_train_famd, y_train=y_train_famd,
    X_test=X_test_famd,   y_test=y_test_famd
)
all_results.append(res_famd)

# Concatenado final con ranking global
results_global = pd.concat(all_results, ignore_index=True)
results_global = results_global.sort_values(
    by=["test_recall_macro","test_f1_macro","test_accuracy"], ascending=[False, False, False]
).reset_index(drop=True)

print("====== RANKING GLOBAL (TEST) por recall_macro / f1_macro ======")
print(results_global[["dataset","combo","n_features","test_accuracy","test_recall_macro",
                      "test_precision_macro","test_f1_macro","test_auc"]].head(20).to_string(index=False))


[Base_em] COMBINACIÓN: DEMOS
Features (6): ['CLASIFICACION', 'SEXO', 'CICLO_VIT', 'EST_CIVIL', 'ESCOLARIDAD', 'CAT_ruralidad 2023']

>> Hiperparámetros óptimos (CV, refit=recall_macro):
{'clf__C': np.float64(0.08777815504719653), 'clf__class_weight': 'balanced', 'clf__penalty': 'l1', 'clf__solver': 'liblinear'}
Mejor score CV (recall_macro): 0.6867

>> MÉTRICAS EN TEST:
Accuracy        : 0.6369
Recall (macro)  : 0.6855
Precision (macro): 0.6783
F1 (macro)      : 0.6364
AUC (OvR/binario): 0.7293

Matriz de confusión (rows=verdad, cols=pred):
[[19064 17602]
 [ 2984 17044]] 

Classification report (TEST):
              precision    recall  f1-score   support

           0     0.8647    0.5199    0.6494     36666
           1     0.4919    0.8510    0.6235     20028

    accuracy                         0.6369     56694
   macro avg     0.6783    0.6855    0.6364     56694
weighted avg     0.7330    0.6369    0.6402     56694


[Base_em] COMBINACIÓN: EMB_full
Features (15): ['MPIO_ANIO_emb

## K Nearest Neighbors

In [151]:
# ============================================================
# RandomizedSearchCV - KNN (KNeighborsClassifier) con múltiples sets
# ============================================================
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    accuracy_score, recall_score, precision_score, f1_score
)
from scipy.stats import randint

from sklearn.neighbors import KNeighborsClassifier

RANDOM_STATE = 42
CV_SPLITS = 5
N_ITER = 60
SCORING = {
    "recall": "recall_macro",
    "f1": "f1_macro",
    "accuracy": "accuracy",
    "roc_auc": "roc_auc_ovr"
}
REFIT = "recall"  # prioriza recall_macro


def build_var_sets(X):
    cols = X.columns
    demos_candidates = [
        'CLASIFICACION','SEXO','CICLO_VIT','EST_CIVIL','ESCOLARIDAD','CAT_ruralidad 2023'
    ]
    DEMOS = [c for c in demos_candidates if c in cols]
    EMB = [c for c in cols if c.startswith('MPIO_ANIO_emb_')]
    FAMD = [c for c in cols if c.startswith('FAMD_')]

    var_sets = {}
    if DEMOS:
        var_sets["DEMOS"] = DEMOS
    if EMB:
        var_sets["EMB_full"] = EMB
        for k in [5, 8, 12, 15]:
            if len(EMB) >= k:
                var_sets[f"EMB_top{k}"] = EMB[:k]
    if FAMD:
        var_sets["FAMD_full"] = FAMD
        for k in [2, 3, 4, 5]:
            if len(FAMD) >= k:
                var_sets[f"FAMD_top{k}"] = FAMD[:k]

    if DEMOS and EMB:
        var_sets["DEMOS+EMB_full"] = DEMOS + EMB
        if len(EMB) >= 8:
            var_sets["DEMOS+EMB_top8"] = DEMOS + EMB[:8]
    if DEMOS and FAMD:
        var_sets["DEMOS+FAMD_full"] = DEMOS + FAMD
        if len(FAMD) >= 3:
            var_sets["DEMOS+FAMD_top3"] = DEMOS + FAMD[:3]

    if not var_sets:
        var_sets["ALL"] = list(cols)
    return var_sets


def _ohe_compatible():
    # OneHotEncoder: compatibilidad sklearn 1.0–1.5+
    try:
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore", sparse=False)


def make_search_knn(n_classes, y_train):
    """
    Crea RandomizedSearchCV con KNeighborsClassifier.
    - KNN requiere escalado de numéricas (distancias).
    - Lidiamos con constraint de n_neighbors <= n_samples.
    """
    pre = ColumnTransformer(transformers=[], remainder="drop")

    knn = KNeighborsClassifier(
        n_neighbors=5,
        weights="uniform",
        algorithm="auto",
        leaf_size=30,
        p=2,                 # 1: Manhattan, 2: Euclid.
        metric="minkowski",
        n_jobs=-1
    )

    pipe = Pipeline(steps=[("pre", pre), ("clf", knn)])

    # Límite superior seguro para k
    max_k = max(2, min(200, len(y_train)))
    # Espacio de hiperparámetros
    param_distributions = {
        "clf__n_neighbors": randint(1, max_k + 1),
        "clf__weights": ["uniform", "distance"],
        "clf__algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
        "clf__leaf_size": randint(15, 60),
        "clf__p": [1, 2],               # Minkowski (Manhattan o Euclid.)
        # metric fijo en 'minkowski' para que p controle distancia
    }

    cv = StratifiedKFold(n_splits=CV_SPLITS, shuffle=True, random_state=RANDOM_STATE)

    search = RandomizedSearchCV(
        estimator=pipe,
        param_distributions=param_distributions,
        n_iter=N_ITER,
        scoring=SCORING,
        refit=REFIT,
        cv=cv,
        n_jobs=-1,
        random_state=RANDOM_STATE,
        verbose=0,
        return_train_score=False
    )
    return search


def fit_with_features_knn(search, X_train, y_train, feat_list):
    Xsub = X_train[feat_list].copy()
    cat_cols = Xsub.select_dtypes(include=['object','category']).columns.tolist()
    num_cols = [c for c in Xsub.columns if c not in cat_cols]

    transformers = []
    if cat_cols:
        transformers.append(("cat", _ohe_compatible(), cat_cols))
    if num_cols:
        transformers.append(("num", StandardScaler(), num_cols))  # KNN: escalar numéricas

    if not transformers:
        # (raro) todo categórico ya OHEado, pero por consistencia:
        transformers.append(("passthrough_all", "passthrough", Xsub.columns.tolist()))

    pre = ColumnTransformer(transformers=transformers, remainder="drop")
    search.estimator.set_params(pre=pre)
    search.fit(Xsub, y_train)
    return search


def evaluate_on_test(estimator, X_test, y_test, feat_list):
    Xsub_test = X_test[feat_list].copy()
    y_pred = estimator.predict(Xsub_test)

    auc = None
    try:
        proba = estimator.predict_proba(Xsub_test)
        auc = roc_auc_score(y_test, proba[:,1]) if proba.shape[1] == 2 \
              else roc_auc_score(y_test, proba, multi_class="ovr")
    except Exception:
        pass

    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred, digits=4)

    return {
        "accuracy": accuracy_score(y_test, y_pred),
        "recall_macro": recall_score(y_test, y_pred, average="macro"),
        "precision_macro": precision_score(y_test, y_pred, average="macro"),
        "f1_macro": f1_score(y_test, y_pred, average="macro"),
        "auc": auc,
        "confusion_matrix": cm,
        "classification_report": report
    }


def run_experiments_knn(dataset_name, X_train, y_train, X_test, y_test):
    results_rows = []
    var_sets = build_var_sets(X_train)
    n_classes = pd.Series(y_train).nunique()

    for combo_name, feat_list in var_sets.items():
        search = make_search_knn(n_classes=n_classes, y_train=y_train)
        search = fit_with_features_knn(search, X_train, y_train, feat_list)
        best_est = search.best_estimator_

        test_metrics = evaluate_on_test(best_est, X_test, y_test, feat_list)

        print("="*110)
        print(f"[{dataset_name}][KNN] COMBINACIÓN: {combo_name}")
        print(f"Features ({len(feat_list)}): {feat_list}")
        print("\n>> Hiperparámetros óptimos (CV, refit=recall_macro):")
        print(search.best_params_)
        print(f"Mejor score CV (recall_macro): {search.best_score_:.4f}\n")

        print(">> MÉTRICAS EN TEST:")
        print(f"Accuracy         : {test_metrics['accuracy']:.4f}")
        print(f"Recall (macro)   : {test_metrics['recall_macro']:.4f}")
        print(f"Precision (macro): {test_metrics['precision_macro']:.4f}")
        print(f"F1 (macro)       : {test_metrics['f1_macro']:.4f}")
        print(f"AUC (OvR/binario): {None if test_metrics['auc'] is None else round(test_metrics['auc'], 4)}\n")

        print("Matriz de confusión (rows=verdad, cols=pred):")
        print(test_metrics["confusion_matrix"], "\n")

        print("Classification report (TEST):")
        print(test_metrics["classification_report"])
        print()

        results_rows.append({
            "dataset": dataset_name,
            "combo": combo_name,
            "n_features": len(feat_list),
            "features": feat_list,
            "best_params": search.best_params_,
            "cv_best_recall_macro": search.best_score_,
            "test_accuracy": test_metrics["accuracy"],
            "test_recall_macro": test_metrics["recall_macro"],
            "test_precision_macro": test_metrics["precision_macro"],
            "test_f1_macro": test_metrics["f1_macro"],
            "test_auc": test_metrics["auc"]
        })

    results_df = pd.DataFrame(results_rows).sort_values(
        by=["dataset", "test_recall_macro", "test_f1_macro"], ascending=[True, False, False]
    ).reset_index(drop=True)

    print("\n" + "#"*110)
    print(f"TOP combinaciones KNN por dataset {dataset_name} (ordenado por recall_macro/f1_macro en TEST):")
    display_cols = ["dataset","combo","n_features","cv_best_recall_macro",
                    "test_accuracy","test_recall_macro","test_precision_macro","test_f1_macro","test_auc"]
    print(results_df[display_cols].head(10).to_string(index=False))
    print("#"*110 + "\n")

    return results_df


# =========================
# Ejecutar para ambos sets (KNN)
# =========================
all_results_knn = []

# 1) EMBEDDINGS
res_em_knn = run_experiments_knn(
    dataset_name="Base_em",
    X_train=X_train_em, y_train=y_train_em,
    X_test=X_test_em,   y_test=y_test_em
)
all_results_knn.append(res_em_knn)

# 2) FAMD
res_famd_knn = run_experiments_knn(
    dataset_name="Base_famd",
    X_train=X_train_famd, y_train=y_train_famd,
    X_test=X_test_famd,   y_test=y_test_famd
)
all_results_knn.append(res_famd_knn)

# Ranking global KNN
results_global_knn = pd.concat(all_results_knn, ignore_index=True).sort_values(
    by=["test_recall_macro","test_f1_macro","test_accuracy"], ascending=[False, False, False]
).reset_index(drop=True)

print("====== RANKING GLOBAL KNN (TEST) por recall_macro / f1_macro ======")
print(results_global_knn[["dataset","combo","n_features","test_accuracy",
                          "test_recall_macro","test_precision_macro","test_f1_macro","test_auc"]]
      .head(20).to_string(index=False))


[Base_em][KNN] COMBINACIÓN: DEMOS
Features (6): ['CLASIFICACION', 'SEXO', 'CICLO_VIT', 'EST_CIVIL', 'ESCOLARIDAD', 'CAT_ruralidad 2023']

>> Hiperparámetros óptimos (CV, refit=recall_macro):
{'clf__algorithm': 'brute', 'clf__leaf_size': 28, 'clf__n_neighbors': 9, 'clf__p': 2, 'clf__weights': 'uniform'}
Mejor score CV (recall_macro): 0.6394

>> MÉTRICAS EN TEST:
Accuracy         : 0.6620
Recall (macro)   : 0.6417
Precision (macro): 0.6362
F1 (macro)       : 0.6380
AUC (OvR/binario): 0.6943

Matriz de confusión (rows=verdad, cols=pred):
[[26067 10599]
 [ 8561 11467]] 

Classification report (TEST):
              precision    recall  f1-score   support

           0     0.7528    0.7109    0.7313     36666
           1     0.5197    0.5725    0.5448     20028

    accuracy                         0.6620     56694
   macro avg     0.6362    0.6417    0.6380     56694
weighted avg     0.6704    0.6620    0.6654     56694


[Base_em][KNN] COMBINACIÓN: EMB_full
Features (15): ['MPIO_ANIO_emb_

## Random Forest

In [150]:
# ============================================================
# RandomizedSearchCV - Random Forest con múltiples sets
# ============================================================
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    accuracy_score, recall_score, precision_score, f1_score
)
from scipy.stats import randint, uniform, loguniform

from sklearn.ensemble import RandomForestClassifier

RANDOM_STATE = 42
CV_SPLITS = 5
N_ITER = 60
SCORING = {
    "recall": "recall_macro",
    "f1": "f1_macro",
    "accuracy": "accuracy",
    "roc_auc": "roc_auc_ovr"
}
REFIT = "recall"  # prioriza recall_macro


def build_var_sets(X):
    cols = X.columns
    demos_candidates = [
        'CLASIFICACION','SEXO','CICLO_VIT','EST_CIVIL','ESCOLARIDAD','CAT_ruralidad 2023'
    ]
    DEMOS = [c for c in demos_candidates if c in cols]
    EMB = [c for c in cols if c.startswith('MPIO_ANIO_emb_')]
    FAMD = [c for c in cols if c.startswith('FAMD_')]

    var_sets = {}
    if DEMOS:
        var_sets["DEMOS"] = DEMOS
    if EMB:
        var_sets["EMB_full"] = EMB
        for k in [5, 8, 12, 15]:
            if len(EMB) >= k:
                var_sets[f"EMB_top{k}"] = EMB[:k]
    if FAMD:
        var_sets["FAMD_full"] = FAMD
        for k in [2, 3, 4, 5]:
            if len(FAMD) >= k:
                var_sets[f"FAMD_top{k}"] = FAMD[:k]

    if DEMOS and EMB:
        var_sets["DEMOS+EMB_full"] = DEMOS + EMB
        if len(EMB) >= 8:
            var_sets["DEMOS+EMB_top8"] = DEMOS + EMB[:8]
    if DEMOS and FAMD:
        var_sets["DEMOS+FAMD_full"] = DEMOS + FAMD
        if len(FAMD) >= 3:
            var_sets["DEMOS+FAMD_top3"] = DEMOS + FAMD[:3]

    if not var_sets:
        var_sets["ALL"] = list(cols)
    return var_sets


def _ohe_compatible():
    # OneHotEncoder: compatibilidad sklearn 1.0–1.5+
    try:
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore", sparse=False)


def make_search_rf(n_classes, y_train=None):
    """
    Crea RandomizedSearchCV con RandomForestClassifier.
    - No escalamos numéricas (árboles no lo requieren).
    - Se explora class_weight para mitigar desbalance.
    """
    pre = ColumnTransformer(transformers=[], remainder="drop")

    rf = RandomForestClassifier(
        n_estimators=500,            # valor medio; se optimizará
        random_state=RANDOM_STATE,
        n_jobs=-1,
        criterion="gini",
        oob_score=False,             # si activas bootstrap=True + oob_score, evita CV leakage
        class_weight=None
    )

    pipe = Pipeline(steps=[("pre", pre), ("clf", rf)])

    # Espacio de hiperparámetros
    # Nota: max_features acepta float (fracción), int, "sqrt", "log2" o None.
    # Usamos una mezcla práctica: categorías + fracción continua.
    param_distributions = {
        "clf__n_estimators": randint(300, 1200),
        "clf__max_depth": randint(3, 40),                 # profundidad moderada
        "clf__min_samples_split": randint(2, 50),
        "clf__min_samples_leaf": randint(1, 30),
        "clf__max_features": ["sqrt", "log2", None, 0.5, 0.7, 0.9],
        "clf__bootstrap": [True, False],
        "clf__class_weight": [None, "balanced", "balanced_subsample"]
    }

    cv = StratifiedKFold(n_splits=CV_SPLITS, shuffle=True, random_state=RANDOM_STATE)

    search = RandomizedSearchCV(
        estimator=pipe,
        param_distributions=param_distributions,
        n_iter=N_ITER,
        scoring=SCORING,
        refit=REFIT,
        cv=cv,
        n_jobs=-1,
        random_state=RANDOM_STATE,
        verbose=0,
        return_train_score=False
    )
    return search


def fit_with_features_rf(search, X_train, y_train, feat_list):
    Xsub = X_train[feat_list].copy()
    cat_cols = Xsub.select_dtypes(include=['object','category']).columns.tolist()
    num_cols = [c for c in Xsub.columns if c not in cat_cols]

    transformers = []
    if cat_cols:
        transformers.append(("cat", _ohe_compatible(), cat_cols))
    if num_cols:
        transformers.append(("num", "passthrough", num_cols))  # árboles: sin escalar

    if not transformers:
        transformers.append(("passthrough_all", "passthrough", Xsub.columns.tolist()))

    pre = ColumnTransformer(transformers=transformers, remainder="drop")
    search.estimator.set_params(pre=pre)
    search.fit(Xsub, y_train)
    return search


def evaluate_on_test(estimator, X_test, y_test, feat_list):
    Xsub_test = X_test[feat_list].copy()
    y_pred = estimator.predict(Xsub_test)

    auc = None
    try:
        proba = estimator.predict_proba(Xsub_test)
        auc = roc_auc_score(y_test, proba[:,1]) if proba.shape[1] == 2 \
              else roc_auc_score(y_test, proba, multi_class="ovr")
    except Exception:
        pass

    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred, digits=4)

    return {
        "accuracy": accuracy_score(y_test, y_pred),
        "recall_macro": recall_score(y_test, y_pred, average="macro"),
        "precision_macro": precision_score(y_test, y_pred, average="macro"),
        "f1_macro": f1_score(y_test, y_pred, average="macro"),
        "auc": auc,
        "confusion_matrix": cm,
        "classification_report": report
    }


def run_experiments_rf(dataset_name, X_train, y_train, X_test, y_test):
    results_rows = []
    var_sets = build_var_sets(X_train)
    n_classes = pd.Series(y_train).nunique()

    for combo_name, feat_list in var_sets.items():
        search = make_search_rf(n_classes=n_classes, y_train=y_train)
        search = fit_with_features_rf(search, X_train, y_train, feat_list)
        best_est = search.best_estimator_

        test_metrics = evaluate_on_test(best_est, X_test, y_test, feat_list)

        print("="*110)
        print(f"[{dataset_name}][RF] COMBINACIÓN: {combo_name}")
        print(f"Features ({len(feat_list)}): {feat_list}")
        print("\n>> Hiperparámetros óptimos (CV, refit=recall_macro):")
        print(search.best_params_)
        print(f"Mejor score CV (recall_macro): {search.best_score_:.4f}\n")

        print(">> MÉTRICAS EN TEST:")
        print(f"Accuracy         : {test_metrics['accuracy']:.4f}")
        print(f"Recall (macro)   : {test_metrics['recall_macro']:.4f}")
        print(f"Precision (macro): {test_metrics['precision_macro']:.4f}")
        print(f"F1 (macro)       : {test_metrics['f1_macro']:.4f}")
        print(f"AUC (OvR/binario): {None if test_metrics['auc'] is None else round(test_metrics['auc'], 4)}\n")

        print("Matriz de confusión (rows=verdad, cols=pred):")
        print(test_metrics["confusion_matrix"], "\n")

        print("Classification report (TEST):")
        print(test_metrics["classification_report"])
        print()

        results_rows.append({
            "dataset": dataset_name,
            "combo": combo_name,
            "n_features": len(feat_list),
            "features": feat_list,
            "best_params": search.best_params_,
            "cv_best_recall_macro": search.best_score_,
            "test_accuracy": test_metrics["accuracy"],
            "test_recall_macro": test_metrics["recall_macro"],
            "test_precision_macro": test_metrics["precision_macro"],
            "test_f1_macro": test_metrics["f1_macro"],
            "test_auc": test_metrics["auc"]
        })

    results_df = pd.DataFrame(results_rows).sort_values(
        by=["dataset", "test_recall_macro", "test_f1_macro"], ascending=[True, False, False]
    ).reset_index(drop=True)

    print("\n" + "#"*110)
    print(f"TOP combinaciones RF por dataset {dataset_name} (ordenado por recall_macro/f1_macro en TEST):")
    display_cols = ["dataset","combo","n_features","cv_best_recall_macro",
                    "test_accuracy","test_recall_macro","test_precision_macro","test_f1_macro","test_auc"]
    print(results_df[display_cols].head(10).to_string(index=False))
    print("#"*110 + "\n")

    return results_df


# =========================
# Ejecutar para ambos sets (RF)
# =========================
all_results_rf = []

# 1) EMBEDDINGS
res_em_rf = run_experiments_rf(
    dataset_name="Base_em",
    X_train=X_train_em, y_train=y_train_em,
    X_test=X_test_em,   y_test=y_test_em
)
all_results_rf.append(res_em_rf)

# 2) FAMD
res_famd_rf = run_experiments_rf(
    dataset_name="Base_famd",
    X_train=X_train_famd, y_train=y_train_famd,
    X_test=X_test_famd,   y_test=y_test_famd
)
all_results_rf.append(res_famd_rf)

# Ranking global RF
results_global_rf = pd.concat(all_results_rf, ignore_index=True).sort_values(
    by=["test_recall_macro","test_f1_macro","test_accuracy"], ascending=[False, False, False]
).reset_index(drop=True)

print("====== RANKING GLOBAL RF (TEST) por recall_macro / f1_macro ======")
print(results_global_rf[["dataset","combo","n_features","test_accuracy",
                         "test_recall_macro","test_precision_macro","test_f1_macro","test_auc"]]
      .head(20).to_string(index=False))


[Base_em][RF] COMBINACIÓN: DEMOS
Features (6): ['CLASIFICACION', 'SEXO', 'CICLO_VIT', 'EST_CIVIL', 'ESCOLARIDAD', 'CAT_ruralidad 2023']

>> Hiperparámetros óptimos (CV, refit=recall_macro):
{'clf__bootstrap': True, 'clf__class_weight': 'balanced_subsample', 'clf__max_depth': 34, 'clf__max_features': 'sqrt', 'clf__min_samples_leaf': 20, 'clf__min_samples_split': 13, 'clf__n_estimators': 809}
Mejor score CV (recall_macro): 0.6872

>> MÉTRICAS EN TEST:
Accuracy         : 0.6358
Recall (macro)   : 0.6852
Precision (macro): 0.6786
F1 (macro)       : 0.6354
AUC (OvR/binario): 0.7301

Matriz de confusión (rows=verdad, cols=pred):
[[18949 17717]
 [ 2931 17097]] 

Classification report (TEST):
              precision    recall  f1-score   support

           0     0.8660    0.5168    0.6473     36666
           1     0.4911    0.8537    0.6235     20028

    accuracy                         0.6358     56694
   macro avg     0.6786    0.6852    0.6354     56694
weighted avg     0.7336    0.6358 

## XGBoost

In [149]:
# ============================================================
# RandomizedSearchCV - XGBoost (XGBClassifier) con múltiples sets
# ============================================================
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    accuracy_score, recall_score, precision_score, f1_score
)
from scipy.stats import loguniform, randint, uniform

from xgboost import XGBClassifier

RANDOM_STATE = 42
CV_SPLITS = 5
N_ITER = 60  # XGB suele requerir un poco más de iteraciones que LogReg
SCORING = {
    "recall": "recall_macro",
    "f1": "f1_macro",
    "accuracy": "accuracy",
    "roc_auc": "roc_auc_ovr"
}
REFIT = "recall"  # prioriza recall_macro

def build_var_sets(X):
    cols = X.columns
    demos_candidates = [
        'CLASIFICACION','SEXO','CICLO_VIT','EST_CIVIL','ESCOLARIDAD','CAT_ruralidad 2023'
    ]
    DEMOS = [c for c in demos_candidates if c in cols]
    EMB = [c for c in cols if c.startswith('MPIO_ANIO_emb_')]
    FAMD = [c for c in cols if c.startswith('FAMD_')]

    var_sets = {}
    if DEMOS:
        var_sets["DEMOS"] = DEMOS
    if EMB:
        var_sets["EMB_full"] = EMB
        for k in [5, 8, 12, 15]:
            if len(EMB) >= k:
                var_sets[f"EMB_top{k}"] = EMB[:k]
    if FAMD:
        var_sets["FAMD_full"] = FAMD
        for k in [2, 3, 4, 5]:
            if len(FAMD) >= k:
                var_sets[f"FAMD_top{k}"] = FAMD[:k]

    if DEMOS and EMB:
        var_sets["DEMOS+EMB_full"] = DEMOS + EMB
        if len(EMB) >= 8:
            var_sets["DEMOS+EMB_top8"] = DEMOS + EMB[:8]
    if DEMOS and FAMD:
        var_sets["DEMOS+FAMD_full"] = DEMOS + FAMD
        if len(FAMD) >= 3:
            var_sets["DEMOS+FAMD_top3"] = DEMOS + FAMD[:3]

    if not var_sets:
        var_sets["ALL"] = list(cols)
    return var_sets

def _ohe_compatible():
    # OneHotEncoder: compatibilidad sklearn 1.0–1.5+
    try:
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore", sparse=False)

def make_search_xgb(n_classes, y_train=None):
    """
    Crea RandomizedSearchCV con XGBClassifier.
    - Si binario, agrega 'scale_pos_weight' al espacio de búsqueda (aprox. balance).
    - No escalamos numéricas (árboles no lo requieren).
    """
    pre = ColumnTransformer(transformers=[], remainder="drop")

    # Base del modelo
    xgb = XGBClassifier(
        objective="multi:softprob" if n_classes > 2 else "binary:logistic",
        num_class=n_classes if n_classes > 2 else None,
        eval_metric="mlogloss" if n_classes > 2 else "logloss",
        tree_method="hist",            # usa 'gpu_hist' si tienes GPU
        booster="gbtree",
        random_state=RANDOM_STATE,
        n_jobs=-1
    )

    pipe = Pipeline(steps=[("pre", pre), ("clf", xgb)])

    # Espacio de hiperparámetros
    param_distributions = {
        "clf__n_estimators": randint(200, 1200),
        "clf__max_depth": randint(3, 12),
        "clf__learning_rate": loguniform(1e-3, 3e-1),
        "clf__subsample": uniform(0.6, 0.4),          # 0.6–1.0
        "clf__colsample_bytree": uniform(0.6, 0.4),   # 0.6–1.0
        "clf__min_child_weight": loguniform(1e-1, 1e2),
        "clf__gamma": loguniform(1e-3, 1e1),
        "clf__reg_alpha": loguniform(1e-6, 1e0),      # L1
        "clf__reg_lambda": loguniform(1e-3, 1e1),     # L2
    }

    # Si es binario: explorar scale_pos_weight
    if n_classes == 2 and y_train is not None:
        # ratio aproximado (neg/pos) para punto de partida
        # evitamos usar valores exactos para no "fijar" el peso; exploramos alrededor
        p = (np.sum(y_train == y_train.unique()[1]))  # conteo de la clase "positiva" (según orden)
        n = len(y_train) - p
        base_spw = (n / max(p, 1))
        # rangos amplios alrededor del ratio
        param_distributions["clf__scale_pos_weight"] = loguniform(max(0.5, base_spw/5), max(1.5, base_spw*5))

    cv = StratifiedKFold(n_splits=CV_SPLITS, shuffle=True, random_state=RANDOM_STATE)

    search = RandomizedSearchCV(
        estimator=pipe,
        param_distributions=param_distributions,
        n_iter=N_ITER,
        scoring=SCORING,
        refit=REFIT,
        cv=cv,
        n_jobs=-1,
        random_state=RANDOM_STATE,
        verbose=0,
        return_train_score=False
    )
    return search

def fit_with_features_xgb(search, X_train, y_train, feat_list):
    Xsub = X_train[feat_list].copy()
    cat_cols = Xsub.select_dtypes(include=['object','category']).columns.tolist()
    num_cols = [c for c in Xsub.columns if c not in cat_cols]

    transformers = []
    if cat_cols:
        transformers.append(("cat", _ohe_compatible(), cat_cols))
    if num_cols:
        transformers.append(("num", "passthrough", num_cols))  # árboles: sin escalar

    if not transformers:
        transformers.append(("passthrough_all", "passthrough", Xsub.columns.tolist()))

    pre = ColumnTransformer(transformers=transformers, remainder="drop")
    search.estimator.set_params(pre=pre)
    search.fit(Xsub, y_train)
    return search

def evaluate_on_test(estimator, X_test, y_test, feat_list):
    Xsub_test = X_test[feat_list].copy()
    y_pred = estimator.predict(Xsub_test)

    auc = None
    try:
        proba = estimator.predict_proba(Xsub_test)
        auc = roc_auc_score(y_test, proba[:,1]) if proba.shape[1] == 2 \
              else roc_auc_score(y_test, proba, multi_class="ovr")
    except Exception:
        pass

    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred, digits=4)

    return {
        "accuracy": accuracy_score(y_test, y_pred),
        "recall_macro": recall_score(y_test, y_pred, average="macro"),
        "precision_macro": precision_score(y_test, y_pred, average="macro"),
        "f1_macro": f1_score(y_test, y_pred, average="macro"),
        "auc": auc,
        "confusion_matrix": cm,
        "classification_report": report
    }

def run_experiments_xgb(dataset_name, X_train, y_train, X_test, y_test):
    results_rows = []
    var_sets = build_var_sets(X_train)
    n_classes = pd.Series(y_train).nunique()

    for combo_name, feat_list in var_sets.items():
        search = make_search_xgb(n_classes=n_classes, y_train=y_train)
        search = fit_with_features_xgb(search, X_train, y_train, feat_list)
        best_est = search.best_estimator_

        test_metrics = evaluate_on_test(best_est, X_test, y_test, feat_list)

        print("="*110)
        print(f"[{dataset_name}][XGB] COMBINACIÓN: {combo_name}")
        print(f"Features ({len(feat_list)}): {feat_list}")
        print("\n>> Hiperparámetros óptimos (CV, refit=recall_macro):")
        print(search.best_params_)
        print(f"Mejor score CV (recall_macro): {search.best_score_:.4f}\n")

        print(">> MÉTRICAS EN TEST:")
        print(f"Accuracy         : {test_metrics['accuracy']:.4f}")
        print(f"Recall (macro)   : {test_metrics['recall_macro']:.4f}")
        print(f"Precision (macro): {test_metrics['precision_macro']:.4f}")
        print(f"F1 (macro)       : {test_metrics['f1_macro']:.4f}")
        print(f"AUC (OvR/binario): {None if test_metrics['auc'] is None else round(test_metrics['auc'], 4)}\n")

        print("Matriz de confusión (rows=verdad, cols=pred):")
        print(test_metrics["confusion_matrix"], "\n")

        print("Classification report (TEST):")
        print(test_metrics["classification_report"])
        print()

        results_rows.append({
            "dataset": dataset_name,
            "combo": combo_name,
            "n_features": len(feat_list),
            "features": feat_list,
            "best_params": search.best_params_,
            "cv_best_recall_macro": search.best_score_,
            "test_accuracy": test_metrics["accuracy"],
            "test_recall_macro": test_metrics["recall_macro"],
            "test_precision_macro": test_metrics["precision_macro"],
            "test_f1_macro": test_metrics["f1_macro"],
            "test_auc": test_metrics["auc"]
        })

    results_df = pd.DataFrame(results_rows).sort_values(
        by=["dataset", "test_recall_macro", "test_f1_macro"], ascending=[True, False, False]
    ).reset_index(drop=True)

    print("\n" + "#"*110)
    print(f"TOP combinaciones XGB por dataset {dataset_name} (ordenado por recall_macro/f1_macro en TEST):")
    display_cols = ["dataset","combo","n_features","cv_best_recall_macro",
                    "test_accuracy","test_recall_macro","test_precision_macro","test_f1_macro","test_auc"]
    print(results_df[display_cols].head(10).to_string(index=False))
    print("#"*110 + "\n")

    return results_df

# =========================
# Ejecutar para ambos sets (XGB)
# =========================
all_results_xgb = []

# 1) EMBEDDINGS
res_em_xgb = run_experiments_xgb(
    dataset_name="Base_em",
    X_train=X_train_em, y_train=y_train_em,
    X_test=X_test_em,   y_test=y_test_em
)
all_results_xgb.append(res_em_xgb)

# 2) FAMD
res_famd_xgb = run_experiments_xgb(
    dataset_name="Base_famd",
    X_train=X_train_famd, y_train=y_train_famd,
    X_test=X_test_famd,   y_test=y_test_famd
)
all_results_xgb.append(res_famd_xgb)

# Ranking global XGB
results_global_xgb = pd.concat(all_results_xgb, ignore_index=True).sort_values(
    by=["test_recall_macro","test_f1_macro","test_accuracy"], ascending=[False, False, False]
).reset_index(drop=True)

print("====== RANKING GLOBAL XGB (TEST) por recall_macro / f1_macro ======")
print(results_global_xgb[["dataset","combo","n_features","test_accuracy",
                          "test_recall_macro","test_precision_macro","test_f1_macro","test_auc"]]
      .head(20).to_string(index=False))


[Base_em][XGB] COMBINACIÓN: DEMOS
Features (6): ['CLASIFICACION', 'SEXO', 'CICLO_VIT', 'EST_CIVIL', 'ESCOLARIDAD', 'CAT_ruralidad 2023']

>> Hiperparámetros óptimos (CV, refit=recall_macro):
{'clf__colsample_bytree': np.float64(0.6093087742943303), 'clf__gamma': np.float64(1.8108143621079664), 'clf__learning_rate': np.float64(0.004991010153018044), 'clf__max_depth': 6, 'clf__min_child_weight': np.float64(11.985911759153476), 'clf__n_estimators': 825, 'clf__reg_alpha': np.float64(0.005938230901955953), 'clf__reg_lambda': np.float64(3.2351025657883583), 'clf__scale_pos_weight': np.float64(1.7417336933723413), 'clf__subsample': np.float64(0.9213923721539394)}
Mejor score CV (recall_macro): 0.6872

>> MÉTRICAS EN TEST:
Accuracy         : 0.6365
Recall (macro)   : 0.6857
Precision (macro): 0.6788
F1 (macro)       : 0.6361
AUC (OvR/binario): 0.731

Matriz de confusión (rows=verdad, cols=pred):
[[19003 17663]
 [ 2943 17085]] 

Classification report (TEST):
              precision    recall  f

## Texmaker

In [155]:
import pandas as pd
import numpy as np

# ===== Helpers =====
def _prep(df: pd.DataFrame, model_name: str) -> pd.DataFrame:
    out = df.copy()
    out["model"] = model_name
    # Orden estándar de columnas
    cols = ["model","dataset","combo","n_features",
            "test_accuracy","test_recall_macro","test_precision_macro","test_f1_macro","test_auc"]
    # Asegurar columnas
    for c in cols:
        if c not in out.columns:
            out[c] = np.nan
    return out[cols]

def _bold_best(df: pd.DataFrame, metric_cols, group_cols=None):
    """Devuelve df (string) con \textbf{} en máximos por columna (global o por grupos)."""
    g = df.groupby(group_cols) if group_cols else [(None, df)]
    out = []
    for _, part in g:
        part_s = part.copy()
        for m in metric_cols:
            try:
                maxv = part[m].astype(float).max()
                part_s[m] = [f"\\textbf{{{v:.4f}}}" if float(v)==maxv else f"{float(v):.4f}" 
                             for v in part[m]]
            except Exception:
                part_s[m] = part[m].apply(lambda v: "--" if pd.isna(v) else f"{v}")
        out.append(part_s)
    return pd.concat(out, ignore_index=True)

def _to_tex(df, caption, label, path, si_cols=("test_accuracy","test_recall_macro","test_precision_macro","test_f1_macro","test_auc")):
    # Formato de números y texto
    df2 = df.copy()
    # n_features como int
    if "n_features" in df2.columns:
        df2["n_features"] = df2["n_features"].astype(int, errors="ignore")
    # Reemplazar NaN AUC por --
    if "test_auc" in df2.columns:
        df2["test_auc"] = df2["test_auc"].apply(lambda v: "--" if pd.isna(v) else v)
    # Orden de columnas cómodo
    cols = [c for c in ["model","dataset","combo","n_features","test_accuracy","test_recall_macro","test_precision_macro","test_f1_macro","test_auc"] if c in df2.columns]
    df2 = df2[cols]
    # LaTeX
    colfmt = "@{} " + " ".join(["l" if c not in si_cols and c!="n_features" else "S" for c in cols]) + " @{}"
    tex = df2.to_latex(index=False, escape=False, column_format=colfmt)
    # Añadir caption/label con entorno table+threeparttable
    wrapper = f"""
\\begin{{table}}[ht]
\\centering
\\begin{{threeparttable}}
\\caption{{{caption}}}
\\label{{{label}}}
{tex}
\\begin{{tablenotes}}\\footnotesize
\\item Métricas en TEST. AUC es OvR si aplica; "--" indica no disponible.
\\end{{tablenotes}}
\\end{{threeparttable}}
\\end{{table}}
"""
    with open(path, "w", encoding="utf-8") as f:
        f.write(wrapper)

# ===== Une resultados de los 4 modelos =====
df_all = pd.concat([
    _prep(results_global, "LogReg"),
    _prep(results_global_rf,     "RF"),
    _prep(results_global_xgb,    "XGB"),
    _prep(results_global_knn,    "KNN"),
], ignore_index=True)

# ===== Tabla A: mejor combinación por modelo =====
best_by_model = (df_all.sort_values(["test_recall_macro","test_f1_macro","test_accuracy"], ascending=False)
                       .groupby("model", as_index=False)
                       .head(1)
                       .sort_values("model"))
best_by_model_fmt = _bold_best(best_by_model, 
                               metric_cols=["test_accuracy","test_recall_macro","test_precision_macro","test_f1_macro","test_auc"])
_to_tex(best_by_model_fmt,
        caption="Mejor combinación por modelo (TEST)",
        label="tab:best_by_model",
        path="best_by_model.tex")

# ===== Tabla B: Top 10 global =====
global_top10 = df_all.sort_values(["test_recall_macro","test_f1_macro","test_accuracy"], ascending=False).head(10)
global_top10_fmt = _bold_best(global_top10, 
                              metric_cols=["test_accuracy","test_recall_macro","test_precision_macro","test_f1_macro","test_auc"])
_to_tex(global_top10_fmt,
        caption="Top 10 global en TEST (ordenado por Recall\\_macro y F1\\_macro)",
        label="tab:global_top10",
        path="global_top10.tex")

# ===== Tabla C: Top 5 por modelo (ejemplo con RF) =====
rf_top5 = (df_all[df_all["model"]=="RF"]
           .sort_values(["test_recall_macro","test_f1_macro","test_accuracy"], ascending=False)
           .head(5))
rf_top5_fmt = _bold_best(rf_top5, 
                         metric_cols=["test_accuracy","test_recall_macro","test_precision_macro","test_f1_macro","test_auc"])
_to_tex(rf_top5_fmt,
        caption="Top 5 — Random Forest (TEST)",
        label="tab:rf_top5",
        path="rf_top5.tex")


In [156]:
best_by_model

,model,dataset,combo,n_features,test_accuracy,test_recall_macro,test_precision_macro,test_f1_macro,test_auc
48,KNN,Base_em,DEMOS+EMB_full,21,0.738967,0.686969,0.717774,0.695240,0.797245
0,LogReg,Base_em,DEMOS+EMB_full,21,0.703372,0.716444,0.698088,0.694997,0.794527
16,RF,Base_em,DEMOS+EMB_top8,14,0.705172,0.720984,0.702049,0.697652,0.798712
32,XGB,Base_em,DEMOS+EMB_top8,14,0.699051,0.721656,0.702682,0.693485,0.799878


In [157]:
global_top10

,model,dataset,combo,n_features,test_accuracy,test_recall_macro,test_precision_macro,test_f1_macro,test_auc
32,XGB,Base_em,DEMOS+EMB_top8,14,0.699051,0.721656,0.702682,0.693485,0.799878
33,XGB,Base_em,DEMOS+EMB_full,21,0.699757,0.721465,0.702457,0.693949,0.799783
16,RF,Base_em,DEMOS+EMB_top8,14,0.705172,0.720984,0.702049,0.697652,0.798712
34,XGB,Base_famd,DEMOS+FAMD_full,11,0.696952,0.720894,0.702097,0.691744,0.796081
17,RF,Base_em,DEMOS+EMB_full,21,0.704960,0.720786,0.701867,0.697445,0.798324
18,RF,Base_famd,DEMOS+FAMD_full,11,0.706054,0.720171,0.701428,0.698006,0.796311
19,RF,Base_famd,FAMD_top4,4,0.692031,0.719616,0.701405,0.687742,0.791529
20,RF,Base_famd,FAMD_top3,3,0.692154,0.719303,0.701053,0.687764,0.791527
21,RF,Base_famd,FAMD_full,5,0.691572,0.719295,0.701138,0.687319,0.791668
22,RF,Base_famd,FAMD_top5,5,0.691572,0.719295,0.701138,0.687319,0.791668


In [166]:
# 1) DataFrames de resultados
# Ejemplos: pon aquí TODO lo que quieras preservar
%store X_train_em
%store y_train_em
%store X_test_em
%store y_test_em

%store global_top10
%store best_by_model

%store X_train_famd
%store y_train_famd
%store X_test_famd
%store y_test_famd

%store results_global
%store results_global_rf
%store results_global_xgb
%store results_global_knn




Stored 'X_train_em' (DataFrame)
Stored 'y_train_em' (Series)
Stored 'X_test_em' (DataFrame)
Stored 'y_test_em' (Series)
Stored 'global_top10' (DataFrame)
Stored 'best_by_model' (DataFrame)
Stored 'X_train_famd' (DataFrame)
Stored 'y_train_famd' (Series)
Stored 'X_test_famd' (DataFrame)
Stored 'y_test_famd' (Series)
Stored 'results_global' (DataFrame)
Stored 'results_global_rf' (DataFrame)
Stored 'results_global_xgb' (DataFrame)
Stored 'results_global_knn' (DataFrame)


In [9]:
15931/(15931+4097)

0.7954363890553225

In [ ]:
[[23741 12925]
 [ 4097 15931]] 

In [7]:
%store -r X_train_em y_train_em X_test_em y_test_em
%store -r X_train_famd y_train_famd X_test_famd y_test_famd
%store -r results_global results_global_rf results_global_xgb results_global_knn
%store -r global_top10 best_by_model


C:\Users\Usuario\AppData\Roaming\Python\Python313\site-packages\IPython\extensions\storemagic.py:148: UserWarning: This is now an optional IPython functionality, using autorestore/X_train_em requires you to install the `pickleshare` library.
  obj = db["autorestore/" + arg]
C:\Users\Usuario\AppData\Roaming\Python\Python313\site-packages\IPython\extensions\storemagic.py:148: UserWarning: This is now an optional IPython functionality, using autorestore/y_train_em requires you to install the `pickleshare` library.
  obj = db["autorestore/" + arg]
C:\Users\Usuario\AppData\Roaming\Python\Python313\site-packages\IPython\extensions\storemagic.py:148: UserWarning: This is now an optional IPython functionality, using autorestore/X_test_em requires you to install the `pickleshare` library.
  obj = db["autorestore/" + arg]
C:\Users\Usuario\AppData\Roaming\Python\Python313\site-packages\IPython\extensions\storemagic.py:148: UserWarning: This is now an optional IPython functionality, using autorest